# 이론 문제

** 각 1점 [총 10점] 모두 O,X 문제 입니다. 10번 문제는 첨부한 수식을 참고해 풀어주세요. **


1. [1점] nn.Linear(a,b)의 a는 입력 차원의 수, b는 출력 차원의 수입니다. -> O
2. [1점] 입력차원이 3개라면 3개의 가중치와 3개의 편향이 필요합니다. -> X (편향은 1개)
3. [1점] optimizer.step()은 기존의 기울기(gradient) 값을 초기화하는 단계에 사용되는 함수이다. -> X (개선하는 단계)
4. [1점] Minibatch Gradient Descent는 전체 데이터를 균일하게 나누어서, 나눈 각각의 데이터를 하나하나 학습시키는 경사하강법을 의미한다. -> O
5. [1점] MSE는 오차 (실제값 - 예측값)들의 합이다. -> X (제곱합)
6. [1점] 학습하고자 하는 데이터의 양이 지나치게 방대한 경우 모든 데이터를 학습할 수는 없다. 이를 해결하고자 minibatch로 데이터를 균일하게 나누어 학습하는데 기존의 Batch gradient descent는 매끄럽게 감소하는 반면 minibatch gradient descent는 다소 거칠게 감소한다. -> O
7. [1점] optimizer = optim.SGD([W, b], lr=0.01) 에서 lr(learning rate)을 크게 설정할수록 W값을 더욱 빠르고 정확하게 찾을 수 있으므로 되도록 크게 설정하는것이 좋다. -> X (발산할지도..)
8. [1점] 다중 선형 회귀에서 x들의 수(샘플의 수 × 특성의 수)가 3X7의 경우 가중치를 W = torch.zeros((3, 1), requires_grad=True)로 선언해야 한다. -> X (7X1)
9. [1점] 보통 Training Loss가 더이상 낮아지지 않을 때 Early Stopping을 하여 Overfitting을 방지한다. -> O
10. [1점] 위 수식에 의하면 hypothesis[0] = log P(x=1;W), (1 – hypothesis[0]) = log P(x=0;W)이다. -> O
    >
    (위의 수식 시그마(i=1부터 m) 중 i = 1인 경우에 해당합니다.)
    >
    (y_train[0] * torch.log(hypothesis[0]) + (1 – y_train[0]) * torch.log(1 – hypothesis[0]))

# 실습문제1 [총 5점]
#### 출제자: 김시현 # 검수자: 나머지
#### 위의 수식을 참고하세요.


```
x_train = torch.FloatTensor([[88],[95],[80],[100],[74]])
y_train = torch.FloatTensor([[80],[94],[85],[99],[73]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train*W+ b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
```


(1) [2점] 위 코드는 optimizer를 사용하여 경사하강법 구현한 것이다.
optimizer를 사용하지 않고 위 코드를 변형하여 경사하강법을 구현하시오.
    (hint) 위에 있는 이미지에 있는 식을 활용하여 구현하시오.

(2) [3점] 위 코드를 nn.Module로 더욱 간단히 구현해 보시오.

In [1]:
import torch
import torch.nn as nn             # 학습 모델 저장 모듈
import torch.nn.functional as F   # 손실함수 저장 모듈
import torch.optim as optim       # 최적화 함수 모듈

### (1)

In [19]:
# 데이터 준비
x_train = torch.FloatTensor([[88],[95],[80],[100],[74]])
y_train = torch.FloatTensor([[80],[94],[85],[99],[73]])

# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

lr=0.01

# 학습
nb_epochs = 100
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x_train*W+ b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    gradient = torch.sum((W * x_train - y_train) * x_train)

    # H(x) 개선
    W = W - lr * gradient

### (2)

In [20]:
# 모델 생성
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [21]:
# 데이터 준비
x_train = torch.FloatTensor([[88],[95],[80],[100],[74]])
y_train = torch.FloatTensor([[80],[94],[85],[99],[73]])

# 모델 선언
model = LinearRegressionModel()

# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

# 학습
nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train*W+ b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

# 실습문제2 [총 10점]
#### 출제자: 송수연  검수자: 나머지

1. [3점] 아래에 주어진 데이터 x_data를 이용하여 내부 배열의 합이 홀수인 경우 1, 짝수인 경우 0으로 라벨링한 y_data를 만드시오. (배열의 합은 코드에서 자동으로 계산되게 하시오.)
		x_data = [[1, 2], [2, 6], [3, 1], [3, 4], [5, 3], [6, 2], [5, 4], [2, 1]]

2. [2점] x_data와 y_data를 float자료형인 텐서 x_train, y_train로 변경하시오.(이때 x_data의 텐서는 [8,2], y_data의 텐서는 [8,1] 차원을 갖게 하시오.)

3. [3점] BinaryClassifier를 수행하는 클래스를 선언하시오. (이때, 입력 차원은 2, 출력 차원은 1이 되게 하시오.)

4. [2점] 아래에 주어진 변수값을 대입하여 100마다, cost를 출력하시오. (nn.Module, nn.Linear, nn.Sigmoid를 이용하시오.)
		epochs = 1000
		learning_rate = 0.01

    (5강 슬라이드 참조)

### (1)

In [26]:
x_data = [[1, 2], [2, 6], [3, 1], [3, 4], [5, 3], [6, 2], [5, 4], [2, 1]]

In [27]:
y_data = [[(x_data[i][0] + x_data[i][1]) % 2] for i in range(len(x_data))]
print(y_data)

[[1], [0], [0], [1], [0], [0], [1], [1]]


### (2)

In [28]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([8, 2])
torch.Size([8, 1])


### (3)

In [31]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [51]:
print(BinaryClassifier().__init__)
print(BinaryClassifier().forward(x_train))

<bound method BinaryClassifier.__init__ of BinaryClassifier(
  (linear): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)>
tensor([[0.6667],
        [0.6139],
        [0.9014],
        [0.8254],
        [0.9558],
        [0.9810],
        [0.9455],
        [0.8268]], grad_fn=<SigmoidBackward0>)


### (4)

In [34]:
# 모델 준비
model = BinaryClassifier()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item(),
        ))

Epoch    0/1000 Cost: 0.840985
Epoch  100/1000 Cost: 0.709105
Epoch  200/1000 Cost: 0.687606
Epoch  300/1000 Cost: 0.677491
Epoch  400/1000 Cost: 0.672026
Epoch  500/1000 Cost: 0.668297
Epoch  600/1000 Cost: 0.665244
Epoch  700/1000 Cost: 0.662508
Epoch  800/1000 Cost: 0.659970
Epoch  900/1000 Cost: 0.657587
Epoch 1000/1000 Cost: 0.655342
